# Experiment #38 - Represent Learn Content By Wiki Articles
We aim here to represente each learn content (and its subdivisions) as a set of wikipedia articles in order to have "blocks" that the learn content would be made from. We may use articles links as priors to help create these representations.

maybe use markov models to find the best links<br>
keep working on this, creating method to elect candidates for each line description <br>
and computing how many words doesn't match anything'

## List of heuristics used
*Removed puntuactions such "!.-" from the text since wikipedia don't often use these in the titles

In [1]:
from wikipydia import wikidb, url

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
#from nltk.util import ngrams

from string import punctuation

from learndata import LearnContents

import networkx as nx

from ThreadPool import ThreadPool

import urllib

#Load learn contents
ld = LearnContents()

#Load wikipedia local db
wdb = wikidb.WikiDb()

stopwords = list(punctuation)

Failed to open href_to_pageid.pickle. Created empty dict.
Failed to open pageid_to_href.pickle. Created empty dict.
Failed to open pageid_to_article.pickle. Created empty dict.


In [2]:
def print_lc_content(lc):
    print(lc)
    for lc in lc:
        print("\t", lc)
        for lc in lc:
            print("\t\t", lc)

In [3]:
top_contents = [lc for lc in ld if lc.parent == None]

### 1. Unroll learning contents sections into a single list

In [4]:
def unroll_lc_contents(lc):
    contents = [lc]
    for lc2 in lc:
        #contents.append(lc2)
        contents += unroll_lc_contents(lc2)
    return contents

In [5]:
unrolled_lc = unroll_lc_contents(ld[0])
unrolled_lc

[Data Visualization with d3.js,
 Chapter 1: Getting Started with d3.js,
 What is d3.js?,
 Setting up a play environment,
 A simple histogram,
 Summary,
 Chapter 2: A Primer on DOM, SVG, and CSS,
 DOM,
 SVG,
 CSS,
 Summary,
 Chapter 3: Making Data Useful,
 Thinking about data functionally,
 Loading data,
 Scales,
 Time,
 Geography,
 Summary,
 Chapter 4: Making Things Move,
 Animating with transitions,
 Interacting with the user,
 Summary,
 Chapter 5: Layouts – d3's Black Magic,
 What are layouts and why should you care,
 Summary,
 Chapter 6: Designing Good Visualizations,
 What is a visualization?,
 Some great examples,
 Summary]

### 2. Generate dict of ngrams for each content

In [6]:
def clear_bad_chars(text):
    """Ensure bad characters (such as '–' that is not a common '-') be removed."""
    return text.encode("ascii", "ignore").decode("ascii")

In [7]:
def get_ngrams(text, max_ngram_size=-1):
    """Tokenize text and them return its ngrams."""
    
    tokens = [word for word in word_tokenize(clear_bad_chars(text.strip())) if word not in stopwords]
    
    if max_ngram_size == -1:
        max_ngram_size = len(tokens)
    
    ngrams_list = list()
    for ngram_size in range(1, max_ngram_size+1):
        for ngram_tuple in nltk.util.ngrams(tokens, ngram_size):
            ngram_text = " ".join(ngram_tuple)
            ngrams_list.append(ngram_text)

    return ngrams_list

In [8]:
content_ngrams_dict = dict()
for lc in unrolled_lc:
    if lc.title not in content_ngrams_dict:
        content_ngrams_dict[lc.title] = get_ngrams(lc.title)
    
content_ngrams_dict

{'A simple histogram': ['A',
  'simple',
  'histogram',
  'A simple',
  'simple histogram',
  'A simple histogram'],
 'Animating with transitions': ['Animating',
  'with',
  'transitions',
  'Animating with',
  'with transitions',
  'Animating with transitions'],
 'CSS': ['CSS'],
 'Chapter 1: Getting Started with d3.js': ['Chapter',
  '1',
  'Getting',
  'Started',
  'with',
  'd3.js',
  'Chapter 1',
  '1 Getting',
  'Getting Started',
  'Started with',
  'with d3.js',
  'Chapter 1 Getting',
  '1 Getting Started',
  'Getting Started with',
  'Started with d3.js',
  'Chapter 1 Getting Started',
  '1 Getting Started with',
  'Getting Started with d3.js',
  'Chapter 1 Getting Started with',
  '1 Getting Started with d3.js',
  'Chapter 1 Getting Started with d3.js'],
 'Chapter 2: A Primer on DOM, SVG, and CSS': ['Chapter',
  '2',
  'A',
  'Primer',
  'on',
  'DOM',
  'SVG',
  'and',
  'CSS',
  'Chapter 2',
  '2 A',
  'A Primer',
  'Primer on',
  'on DOM',
  'DOM SVG',
  'SVG and',
  'and C

### 3. Get a set of all ngram generated

In [9]:
ngrams_list = list()
for ngrams in content_ngrams_dict.values():
    ngrams_list += ngrams
ngrams_set = set(ngrams_list)
ngrams_set

{"'s",
 "'s Black",
 "'s Black Magic",
 '1',
 '1 Getting',
 '1 Getting Started',
 '1 Getting Started with',
 '1 Getting Started with d3.js',
 '2',
 '2 A',
 '2 A Primer',
 '2 A Primer on',
 '2 A Primer on DOM',
 '2 A Primer on DOM SVG',
 '2 A Primer on DOM SVG and',
 '2 A Primer on DOM SVG and CSS',
 '3',
 '3 Making',
 '3 Making Data',
 '3 Making Data Useful',
 '4',
 '4 Making',
 '4 Making Things',
 '4 Making Things Move',
 '5',
 '5 Layouts',
 '5 Layouts d3',
 "5 Layouts d3 's",
 "5 Layouts d3 's Black",
 "5 Layouts d3 's Black Magic",
 '6',
 '6 Designing',
 '6 Designing Good',
 '6 Designing Good Visualizations',
 'A',
 'A Primer',
 'A Primer on',
 'A Primer on DOM',
 'A Primer on DOM SVG',
 'A Primer on DOM SVG and',
 'A Primer on DOM SVG and CSS',
 'A simple',
 'A simple histogram',
 'Animating',
 'Animating with',
 'Animating with transitions',
 'Black',
 'Black Magic',
 'CSS',
 'Chapter',
 'Chapter 1',
 'Chapter 1 Getting',
 'Chapter 1 Getting Started',
 'Chapter 1 Getting Started w

### 4. Try to get a wikipedia article for each ngram

In [10]:
def get_text_article_candidates(text):
    """Get the wikiarticles candidates to describe a piece of text."""
    
    candidates = list()
    
    for ngram_text in get_ngrams(text):
        try:
            art, _ = wdb.get_article_by_href(url.UnquotedURL(ngram_text))
            candidates.append(art)
            #print(ngram_text, " -> ", art.title())
        except:
            continue
            #print("Not found: ", ngram_text)
    return candidates

In [11]:
wiki_articles = dict()
def get_wiki_article(href):
    try:
        art, _ = wdb.get_article_by_href(url.UnquotedURL(href))
        wiki_articles[href] = art
    except:
        pass
    

pool = ThreadPool(10)
%time pool.map(get_wiki_article, ngrams_set)
%time pool.wait_completion()
del pool
wdb.save()
wiki_articles

Wall time: 20.5 s
Wall time: 2.12 s


{"'s": 's,
 '1': 1,
 '2': 2,
 '2 A': 2A,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 'A': A,
 'Animating': Animation,
 'Black': Black,
 'Black Magic': Black magic,
 'CSS': Cascading Style Sheets,
 'Chapter': Chapter,
 'Chapter 1': Chapter One,
 'Chapter 2': Chapter Two,
 'Chapter 3': Chapter Three,
 'Chapter 4': Chapter Four,
 'Chapter 5': Chapter Five,
 'Chapter 6': Chapter Six,
 'DOM': Dom,
 'Data': Data,
 'Data Visualization': Data visualization,
 'Designing': Design,
 'Geography': Geography,
 'Getting': Receiving,
 'Getting Started': Getting Started,
 'Good': Good,
 'Interacting': Interaction,
 'Layouts': Layout,
 'Loading': Loading,
 'Magic': Magic,
 'Making': Making,
 'Move': Move,
 'Primer': Primer,
 'SVG': Scalable Vector Graphics,
 'Scales': Scale,
 'Setting': Setting,
 'Some': Some,
 'Summary': Summary,
 'Things': The Things,
 'Thinking': Thought,
 'Time': Time,
 'Useful': Utility,
 'Visualization': Visualization,
 'What': What,
 'a': A,
 'about': About,
 'and': And,
 'are': ARE,
 '

### 5. Construct graph of candidate articles and links

In [18]:
def get_wikipedia_page_url(title):
    return urllib.parse.quote(title.replace(" ", "_"))
#get_wikipedia_page_url("lucas vieira c++")

In [21]:
#Get the disambiguation articles link and put them into a dict mapping from links to ngram that it represents
disamb_articles = dict()

for ngram, article in wiki_articles.items():
    if not article._is_disamb_page:
        continue
        
    for link_href, link_text in article.links():
        disamb_articles[link_href] = ngram

In [25]:
#Get nodes allowed to be at the graph
#Those are the articles candidates and the disambiguation links
allowed_nodes = list(disamb_articles.keys())

for cand in wiki_articles.values():
    if not cand._is_disamb_page:
        allowed_nodes.append(get_wikipedia_page_url(cand.title())) 

In [26]:
#Create graph
graph = nx.DiGraph()

for candidate in wiki_articles.values():
    if candidate._is_disamb_page:
        continue
    
    cand_url = get_wikipedia_page_url(candidate.title())
    for link_href, link_text in candidate.links():
        if link_href in allowed_nodes:
            graph.add_edge(cand_url, link_href)

In [31]:
#list(nx.weakly_connected_components(graph))

In [ ]:
keep working on handling desambiguation pages
then elect nodes that will be used
and elect nodes for each learn content part

In [ ]:
graph.edges()

In [25]:
lc_contents = unroll_lc_contents(ld[0])
candidates = list()
for i, lc in enumerate(lc_contents):
    print("Working on {}/{}".format(i+1, len(lc_contents)))
    candidates += get_text_article_candidates(lc.title)
    
wdb.save()

Working on 1/29
Working on 2/29
Working on 3/29
Working on 4/29
Working on 5/29
Working on 6/29
Working on 7/29
Working on 8/29
Working on 9/29
Working on 10/29
Working on 11/29
Working on 12/29
Working on 13/29
Working on 14/29
Working on 15/29
Working on 16/29
Working on 17/29
Working on 18/29
Working on 19/29
Working on 20/29
Working on 21/29
Working on 22/29
Working on 23/29
Working on 24/29
Working on 25/29
Working on 26/29
Working on 27/29
Working on 28/29
Working on 29/29


In [60]:
#candidates

In [48]:
#Generate graph
graph = nx.DiGraph()

candidates_urls = [get_wikipedia_page_url(cand.title()) for cand in candidates]

for cand_url, candidate in zip(candidates_urls, candidates):
    for link_href, link_text in candidate.links():
        if link_href in candidates_urls:
            graph.add_edge(cand_url, link_href)

In [61]:
print_lc_content(ld[0])

Data Visualization with d3.js
	 Chapter 1: Getting Started with d3.js
		 What is d3.js?
		 Setting up a play environment
		 A simple histogram
		 Summary
	 Chapter 2: A Primer on DOM, SVG, and CSS
		 DOM
		 SVG
		 CSS
		 Summary
	 Chapter 3: Making Data Useful
		 Thinking about data functionally
		 Loading data
		 Scales
		 Time
		 Geography
		 Summary
	 Chapter 4: Making Things Move
		 Animating with transitions
		 Interacting with the user
		 Summary
	 Chapter 5: Layouts – d3's Black Magic
		 What are layouts and why should you care
		 Summary
	 Chapter 6: Designing Good Visualizations
		 What is a visualization?
		 Some great examples
		 Summary


In [85]:
#sorted(candidates[4].links())

In [86]:
#graph.edges()

In [90]:
list(nx.weakly_connected_components(graph))

[{'1',
  '2',
  '3',
  '4',
  '5',
  '6',
  'Animation',
  'Cascading_Style_Sheets',
  'D3',
  'D3.js',
  'Data',
  'Data_visualization',
  'Histogram',
  'Scalable_Vector_Graphics',
  'Time',
  'Visualization'},
 {'Black', 'Black_magic', 'Geography', 'Interaction'},
 {'The', 'You'},
 {'Design', 'Thought'},
 {'Good', 'Utility'}]

In [77]:
for node_name in list(nx.weakly_connected_components(graph))[0]:
    print(node_name, len(graph.in_edges(node_name)))

D3 0
4 2
Visualization 0
6 1
Time 1
5 2
1 1
Histogram 1
2 1
D3.js 2
Animation 1
Data 2
3 3
Cascading_Style_Sheets 2
Data_visualization 3
Scalable_Vector_Graphics 2


In [87]:
#graph['D3.js']

In [88]:
#graph.edges()

In [47]:
#sorted(nx.in_degree_centrality(graph).items(), key=lambda a:a[1], reverse=True)

In [112]:
import requests

def _download_page_data(page, lang, timeout):
    """Function to retrieve a wikipedia page in html form, with its sections"""
    #import exceptions

    #assert isinstance(page, URL)

    # https://en.wikipedia.org/w/api.php?action=parse&redirects&page=fluid_mechanics

    req_params = [
        'action=parse',
        'redirects',
        'format=json',
        'prop=text|displaytitle|categories',
        'page=' + page
    ]

    wikipedia_api_url = "https://" + lang + ".wikipedia.org/w/api.php?" + "&".join(req_params)

    try:
        page_data = requests.get(wikipedia_api_url, timeout=timeout).json()
    except requests.exceptions.ConnectTimeout:
        raise exceptions.PageRequestTimeout(page, lang, timeout)

    #If the object parse is not in the json object, page does not exists
    if not 'parse' in page_data:
        raise exceptions.PageDoesNotExists(page, lang)

    page_title = page_data['parse']['title']
    page_id = page_data['parse']['pageid']
    page_html = page_data['parse']['text']['*']

    #return page, page_title, page_id, page_html
    return page_data

In [129]:
a = _download_page_data("mqtt", "en", 60)['parse']['categories']

In [130]:
a

[{'*': 'CS1_maint:_Multiple_names:_authors_list', 'hidden': '', 'sortkey': ''},
 {'*': 'Application_layer_protocols', 'sortkey': ''},
 {'*': 'Data_transmission', 'sortkey': ''},
 {'*': 'IBM_WebSphere', 'sortkey': 'MQ'},
 {'*': 'Message-oriented_middleware', 'sortkey': ''},
 {'*': 'Network_protocols', 'sortkey': ''},
 {'*': 'Telemetry', 'sortkey': ''}]

In [131]:
def is_disambiguation_page(categories):
    for cat in categories:
        if cat['*'] == 'Disambiguation_pages':
            return True
    return False

In [132]:
is_disambiguation_page(a)

False

In [91]:
we need to check how to handle disambiguation pages such
https://en.wikipedia.org/wiki/Dom
we could first compare the links that the page return and check if it matches the graph, if so, get the
N more matched article links

SyntaxError: invalid syntax (<ipython-input-91-4f244b6b473b>, line 1)